# Simple Recommender

Jednostavni sistemi za preporuku svima korisnicima pruzaju iste preporuke, ne uzimajuci u obzir to da korisnici imaju razlicite ukuse i na primjer daju prednos razlicitim zanrovima. Ovo predstavlja manu jednostavnih sistema, zbog koje
oni nisu pouzdanii  rijetko se koriste, iako se lako implementiraju.

In [1]:
import pandas as pd
import numpy as np

import warnings; warnings.simplefilter('ignore')

In [2]:
rating_df = pd.read_csv('dataset/ratings.csv') 
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movies_df = pd.read_csv('dataset/movies.csv') 
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies_df['count_votes']=rating_df.groupby('movieId')['movieId'].transform('count')
movies_df['average_votes']=rating_df.groupby('movieId')['rating'].transform('mean')
movies_df.head()

,movieId,title,genres,count_votes,average_votes
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,52,3.259615
2,3,Grumpier Old Men (1995),Comedy|Romance,102,3.946078
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,203,3.975369
4,5,Father of the Bride Part II (1995),Comedy,204,4.237745


Za odredjivanje najpopularnijih sistema koristi se IMDB *weighted rating* formula:

*weighted rating* (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

gdje je
* *v* broj ocjena svakog filma
* *m* najmanji broj dobijenih ocjena potrebnih da bi film bio potencijalno preporucen
* *R* prosjecna ocjena filma
* *C* srednja vrijednost svih prosjecnih ocjena filmova

In [5]:
C=movies_df.average_votes.mean()
C

3.5143119130009497

In [6]:
m=movies_df.count_votes.quantile(0.98)
m

251.0

In [7]:
weighted_r=movies_df[movies_df['count_votes']>m]
weighted_r=weighted_r.reset_index()
weighted_r=weighted_r.drop(['index'], axis=1)
weighted_r.head()

,movieId,title,genres,count_votes,average_votes
0,17,Sense and Sensibility (1995),Drama|Romance,307,4.197068
1,21,Get Shorty (1995),Comedy|Crime|Thriller,329,4.164134
2,38,It Takes Two (1995),Children|Comedy,279,4.161290
3,196,Species (1995),Horror|Sci-Fi,278,4.192446
4,270,Love Affair (1994),Drama|Romance,317,4.429022


In [8]:
def weighted_rating(x):
    v = x['count_votes']
    R = x['average_votes']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [9]:
weighted_r['wr']=weighted_r.apply(weighted_rating, axis=1) # dodavanje kolone koja oznacava weighted rating

In [10]:
weighted_r=weighted_r.drop(['count_votes', 'average_votes'], axis=1)
weighted_r.head()

,movieId,title,genres,wr
0,17,Sense and Sensibility (1995),Drama|Romance,3.889950
1,21,Get Shorty (1995),Comedy|Crime|Thriller,3.882918
2,38,It Takes Two (1995),Children|Comedy,3.854891
3,196,Species (1995),Horror|Sci-Fi,3.870685
4,270,Love Affair (1994),Drama|Romance,4.024810


In [11]:
weighted_r = weighted_r.sort_values(by='wr', ascending=False)
weighted_r.head()

,movieId,title,genres,wr
59,5493,In Like Flint (1967),Action|Adventure|Comedy,4.02481
28,1923,There's Something About Mary (1998),Comedy|Romance,4.02481
155,128488,Wild Card (2015),Action|Crime|Drama|Thriller,4.02481
36,2115,Indiana Jones and the Temple of Doom (1984),Action|Adventure|Fantasy,4.02481
154,127130,Mistress America (2015),Comedy,4.02481


## Prikaz 10 najpopularnijih filmova na osnovu *weighted rating* 

In [12]:
top_movies=pd.DataFrame(weighted_r.title.drop_duplicates())
top_movies=top_movies.reset_index()
top_movies=top_movies.drop(['index'], axis=1)
top_movies.head(10)

,title
0,In Like Flint (1967)
1,There's Something About Mary (1998)
2,Wild Card (2015)
3,Indiana Jones and the Temple of Doom (1984)
4,Mistress America (2015)
5,No Small Affair (1984)
6,Stalingrad (2013)
7,Shakespeare in Love (1998)
8,Kung Pow: Enter the Fist (2002)
9,Splinter (2008)
